In [ ]:
import gzip
import math
import pickle

import seaborn as sns

import matplotlib.pyplot as plt
from rdkit.Chem import PandasTools, Crippen


import warnings
import numpy as np 
import pandas as pd
from rdkit import Chem
from rdkit import rdBase
from rdkit import DataStructs
rdBase.DisableLog('rdApp.error')
warnings.filterwarnings("ignore")

import argparse
from rdkit.Chem import PandasTools
from rdkit.Chem.rdMolDescriptors import GetMorganFingerprintAsBitVect
import os

import logging
from rdkit.Chem import PandasTools, QED, Descriptors, rdMolDescriptors
import re

import pandas as pd
from rdkit import Chem
from rdkit.Chem import Draw
import matplotlib.pyplot as plt

import os

# Save the original working directory
original_cwd = os.getcwd()

# Change to the directory where SA_score.pkl.gz and mol_metrics are located
os.chdir()


from qed_drd2.mol_metrics import *
from qed_drd2.evaluation import *

os.chdir(original_cwd)


In [ ]:
def load_smiles_from_csv(file_path, training_smiles):
    generated_smiles = []
    with open(file_path, "r") as f:
        for line in f.readlines()[1:]:
            generated_smiles.append(line.strip())

    # SMILES to molecules
    generated_mols = np.array([Chem.MolFromSmiles(s) for s in generated_smiles if len(s.strip())])

    valid_smiles = [Chem.MolToSmiles(mol) for mol in generated_mols if mol != None and mol.GetNumAtoms() > 1 and Chem.MolToSmiles(mol) != ' ']
    unique_smiles = list(set(valid_smiles))
    novel_smiles = [smi for smi in unique_smiles if smi not in training_smiles]

    validity = len(valid_smiles)/len(generated_smiles)
    if len(valid_smiles) == 0:
        valid_smiles.append('c1ccccc1')
    if len(unique_smiles) == 0:
        unique_smiles.append('c1ccccc1')
    uniqueness = len(unique_smiles)/len(valid_smiles)
    novelty = len(novel_smiles)/len(unique_smiles)
    diversity = mean_diversity(novel_smiles)
    total = len(novel_smiles)/len(generated_smiles)

    print('\nResults Report:')
    print('*'*80)
    print("Total Mols:   {}".format(len(generated_mols)))
    print("Validity:     {}    ({:.2f}%)".format(len(valid_smiles), validity*100))
    print("Uniqueness:   {}    ({:.2f}%)".format(len(unique_smiles), uniqueness*100))
    print("Novelty:      {}    ({:.2f}%)".format(len(novel_smiles), novelty*100))
    print("Total:               {:.2f}%".format(total*100))
    print("Diversity:           {:.2f}".format(diversity))

    




    return novel_smiles

In [ ]:
def calculate_properties(smiles_list):
    """
    Calculate QED, solubility, and SA scores for a list of SMILES strings.
    Args:
        smiles_list (list): List of SMILES strings.
    Returns:
        pd.DataFrame: DataFrame with columns ['QED', 'Solubility', 'SA'].
    """
    mols = pd.Series(smiles_list).apply(Chem.MolFromSmiles).dropna()
    mols = mols[mols.apply(lambda x: x.GetNumAtoms() > 1)]
    
    if mols.empty:
        return pd.DataFrame(columns=['SMILES', 'QED', 'Solubility', 'SA', 'DRD2'])
    
    
    smiles_df = pd.DataFrame({
        'SMILES': mols.apply(Chem.MolToSmiles),
        'QED': mols.apply(QED.qed),
        'Solubility': mols.apply(solubility),
        'SA': mols.apply(SA),
        'DRD2': mols.apply(DRD2)

    })
    # Compute mean values
    mean_values = smiles_df[['QED', 'Solubility', 'SA', 'DRD2']].mean()
    
    print(mean_values)
    
    return smiles_df

In [ ]:
def top_k_smiles(smiles_df, property_name, k):

    if property_name not in smiles_df.columns:
        raise ValueError(f"Property '{property_name}' not found in the DataFrame.")

    top_k_df = smiles_df.nlargest(k, property_name)
    return top_k_df


In [ ]:
generated_path = "qed_drd2/Results/QED_DRD2/qed_w=0.3_drd2_w=0.7/Alphas/0.5/logs/best_QED_DRD2_generated_data.csv"



# Path to the combined properties CSV file
csv_file_path = 'qed_drd2/chembl_selected_combined_properties.csv'

combined_data_df = pd.read_csv(csv_file_path)


training_smiles = combined_data_df['SMILES'].tolist()


 




In [ ]:
# Load and process datasets
novel_smiles_list = load_smiles_from_csv(generated_path, training_smiles)




In [ ]:

smiles_df = calculate_properties(novel_smiles_list)



In [ ]:
top3_drd2_df = top_k_smiles(smiles_df, 'DRD2', 3)

# Write to CSV
top3_drd2_df.to_csv('top3_DRD2.csv', index=False)


In [ ]:
top3_drd2_df